#### Creating `product` Delta table using DDL Script

In [0]:
%fs ls /user/hive/warehouse/product

path,name,size,modificationTime
dbfs:/user/hive/warehouse/product/_delta_log/,_delta_log/,0,0


In [0]:
%fs rm -r /user/hive/warehouse/product

res1: Boolean = true

In [0]:
%sql
DROP TABLE IF EXISTS product;
CREATE  TABLE IF NOT EXISTS `product` (
  `productCode` STRING,
  `productName` STRING,
  `productLine` STRING,
  `productScale` STRING,
  `productVendor` STRING,
  `productDescription` STRING,
  `quantityInStock` INT,
  `buyPrice` DOUBLE,
  `MSRP` DOUBLE,
  `createdDate` TIMESTAMP,
  `createdBy` STRING,
  `updatedDate` TIMESTAMP,
  `updatedBy` STRING)
USING delta;

##### Creating `pyspark Schema` for product table to avoid datatypes issues while reading data.

In [0]:
from pyspark.sql.types import StringType,IntegerType,DoubleType,StructField,StructType
product_schema= StructType([StructField("productCode",StringType(),True),
                            StructField("productName",StringType(),True),
                            StructField("productLine",StringType(),True),
                            StructField("productScale",StringType(),True),
                            StructField("productVendor",StringType(),True),
                            StructField("productDescription",StringType(),True),
                            StructField("quantityInStock",IntegerType(),True),
                            StructField("buyPrice",DoubleType(),True),
                            StructField("MSRP",DoubleType(),True)])

In [0]:
product_badpath="/tmp/productsBadPath/"
product_source_file="dbfs:/FileStore/products/product.txt"
product_delimiter="\t"
product_delta_file="dbfs:/FileStore/products/product_delta.txt"

In [0]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/master/poc_product/product.txt","/tmp/product.txt")



Out[12]: ('/tmp/product.txt', <http.client.HTTPMessage at 0x7f26acddecd0>)

In [0]:
%sh
ls /tmp

Rserv
Rtmp5kSE6D
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
product.txt
python_lsp_logs
systemd-private-8927e32899474482ba02ddf53f93f375-apache2.service-CT8Qaf
systemd-private-8927e32899474482ba02ddf53f93f375-ntp.service-zG260h
systemd-private-8927e32899474482ba02ddf53f93f375-systemd-logind.service-tpValg
systemd-private-8927e32899474482ba02ddf53f93f375-systemd-resolved.service-MOSj9h
tmp.urlF6ZWtFs


In [0]:
%fs ls dbfs:/FileStore/products/

path,name,size,modificationTime
dbfs:/FileStore/products/product.txt,product.txt,29497,1706779608000
dbfs:/FileStore/products/product_delta.txt,product_delta.txt,861,1706779611000


In [0]:
dbutils.fs.mv("file:/tmp/product.txt","dbfs:/FileStore/products/product.txt")

Out[14]: True

In [0]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/master/poc_product/product_delta.txt","/tmp/product_delta.txt")
dbutils.fs.mv("file:/tmp/product_delta.txt","dbfs:/FileStore/products/product_delta.txt")
print('product files copied to ',product_source_file)
print('product delta files copied to ',product_delta_file)

product files copied to  dbfs:/FileStore/products/product.txt
product delta files copied to  dbfs:/FileStore/products/product_delta.txt


In [0]:
%fs head dbfs:/FileStore/products/product_delta.txt

productCode	productName	productLine	productScale	productVendor	productDescription	quantityInStock	buyPrice	MSRP
S10_1678	1969 Harley Davidson Ultimate Chopper	Motorcycles	01:10	Min Lin Diecast	"Updated Quantity, Buy Price and MSRP"	9833	51.81	99
S10_1949	1952 Alpine Renault 1300	Classic Cars	01:10	Classic Metal Creations	"Updated Quantity, Buy Price and MSRP"	8000	99	235
S10_9001	1952 Alpine Renault 1500	Classic Cars	01:10	Classic Metal Creations	"Turnable front wheels, steering function, detailed interior, detailed engine, opening hood, opening trunk, opening doors, and detailed chassis."	7305	102.58	315.3
S10_9002	1952 Alpine Renault 1600	Classic Cars	01:10	Classic Metal Creations	"Turnable front wheels, steering function, detailed interior, detailed engine, opening hood, opening trunk, opening doors, and detailed chassis."	7305	110.58	615.3

In [0]:
%fs head dbfs:/FileStore/products/product_delta.txt

productCode	productName	productLine	productScale	productVendor	productDescription	quantityInStock	buyPrice	MSRP
S10_1678	1969 Harley Davidson Ultimate Chopper	Motorcycles	01:10	Min Lin Diecast	"Updated Quantity, Buy Price and MSRP"	9833	51.81	99
S10_1949	1952 Alpine Renault 1300	Classic Cars	01:10	Classic Metal Creations	"Updated Quantity, Buy Price and MSRP"	8000	99	235
S10_9001	1952 Alpine Renault 1500	Classic Cars	01:10	Classic Metal Creations	"Turnable front wheels, steering function, detailed interior, detailed engine, opening hood, opening trunk, opening doors, and detailed chassis."	7305	102.58	315.3
S10_9002	1952 Alpine Renault 1600	Classic Cars	01:10	Classic Metal Creations	"Turnable front wheels, steering function, detailed interior, detailed engine, opening hood, opening trunk, opening doors, and detailed chassis."	7305	110.58	615.3

In [0]:
%fs head dbfs:/FileStore/products/product.txt

productCode	productName	productLine	productScale	productVendor	productDescription	quantityInStock	buyPrice	MSRP
S10_1678	1969 Harley Davidson Ultimate Chopper	Motorcycles	01:10	Min Lin Diecast	"This replica features working kickstand, front suspension, gear-shift lever, footbrake lever, drive chain, wheels and steering. All parts are particularly delicate due to their precise scale and require special care and attention."	7933	48.81	95.7
S10_1949	1952 Alpine Renault 1300	Classic Cars	01:10	Classic Metal Creations	"Turnable front wheels, steering function, detailed interior, detailed engine, opening hood, opening trunk, opening doors, and detailed chassis."	7305	98.58	214.3
S10_2016	1996 Moto Guzzi 1100i	Motorcycles	01:10	Highway 66 Mini Classics	"Official Moto Guzzi logos and insignias, saddle bags located on side of motorcycle, detailed engine, working steering, working suspension, two leather seats, luggage rack, dual exhaust pipes, small saddle bag located on handle bars, two-tone paint with chrome accents, superior die-cast detail , rotating wheels , working kick stand, diecast metal with plastic parts and baked enamel finish."	6625	68.99	118.94
S10_4698	2003 Harley-Davidson Eagle Drag Bike	Motorcycles	01:10	Red Start Diecast	"Model features, official Harley Davidson logos and insignias, detachable rear wheelie bar, heavy diecast metal with resin parts, authentic multi-color tampo-printed graphics, separate engine drive belts, free-turning front fork, rotating tires and rear racing slick, certificate of authenticity, detailed engine, display stand\r\n, precision diecast replica, baked enamel finish, 1:10 scale model, removable fender, seat and tank cover piece for displaying the superior detail of the v-twin engine"	5582	91.02	193.66
S10_4757	1972 Alfa Romeo GTA	Classic Cars	01:10	Motor City Art Classics	"Features include: Turnable front wheels, steering function, detailed interior, detailed engine, opening hood, opening trunk, opening doors, and detailed chassis."	3252	85.68	136
S10_4962	1962 LanciaA Delta 16V	Classic Cars	01:10	Second Gear Diecast	"Features include: Turnable front wheels, steering function, detailed interior, detailed engine, opening hood, opening trunk, opening doors, and detailed chassis."	6791	103.42	147.74
S12_1099	1968 Ford Mustang	Classic Cars	01:12	Autoart Studio Design	"Hood, doors and trunk all open to reveal highly detailed interior features. Steering wheel actually turns the front wheels. Color dark green."	68	95.34	194.57
S12_1108	2001 Ferrari Enzo	Classic Cars	01:12	Second Gear Diecast	"Turnable front wheels, steering function, detailed interior, detailed engine, opening hood, opening trunk, opening doors, and detailed chassis."	3619	95.59	207.8
S12_1666	1958 Setra Bus	Trucks and Buses	01:12	Welly Diecast Productions	"Model features 30 windows, skylights & glare resistant glass, working steering system, original logos"	1579	77.9	136.67
S12_2823	2002 Suzuki XREO	Motorcycles	01:12	Unimax Art Galleries	"Official logos and insignias, saddle bags located on side of motorcycle, detailed engine, working steering, working suspension, two leather seats, luggage rack, dual exhaust pipes, small saddle bag located on handle bars, two-tone paint with chrome accents, superior die-cast detail , rotating wheels , working kick stand, diecast metal with plastic parts and baked enamel finish."	9997	66.27	150.62
S12_3148	1969 Corvair Monza	Classic Cars	01:18	Welly Diecast Productions	"1:18 scale die-cast about 10\"" long doors open, hood opens, trunk opens and wheels roll"	6906	89.14	151.08
S12_3380	1968 Dodge Charger	Classic Cars	01:12	Welly Diecast Productions	"1:12 scale model of a 1968 Dodge Charger. Hood, doors and trunk all open to reveal highly detailed interior features. Steering wheel actually turns the front wheels. Color black"	9123	75.16	117.44
S12_3891	1969 Ford Falcon	Classic Cars	01:12	Second Gear Diecast	"Turnable front wheels, steering function, detailed interior, detailed engine, opening hood,